In [18]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import cv2 as cv
import math
import sys
import os

In [5]:
vid_index = 14

angle_t = np.load(f"/media/nevin/Trash Games1/Downloads/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b_2018-07-27--06-03-57/{vid_index}/processed_log/CAN/steering_angle/t")
angle_v = np.load(f"/media/nevin/Trash Games1/Downloads/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b_2018-07-27--06-03-57/{vid_index}/processed_log/CAN/steering_angle/value")
speed = np.load(f"/media/nevin/Trash Games1/Downloads/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b_2018-07-27--06-03-57/{vid_index}/processed_log/CAN/speed/value")
frame_times = np.load(f"/media/nevin/Trash Games1/Downloads/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b_2018-07-27--06-03-57/{vid_index}/global_pose/frame_times")
vid = cv.VideoCapture(f"/media/nevin/Trash Games1/Downloads/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b_2018-07-27--06-03-57/{vid_index}/video.hevc")
wheel = cv.imread("/home/nevin/Downloads/wheel.png")

In [3]:
print(angle_t.shape)
print(angle_v.shape)
print(frame_times[0])

(4973,)
(4973,)
9674.089929


In [17]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR)
    return result


def find_nearest(array, value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx-1
    
    else:
        return idx



In [15]:
if vid.isOpened() == False:
    print("error opening video")

i = 0

while vid.isOpened():
    status, frame = vid.read()

    if status == False:
        break

    # rot_idx = angle_t.index(frame_times[i])

    rot_idx = find_nearest(angle_t, frame_times[i])
    frame = cv.putText(frame, str(angle_v[rot_idx]), (0, 100), cv.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 2, cv.LINE_AA)
    rot_wheel = rotate_image(wheel, angle_v[rot_idx])

    # frame = np.concatenate((frame, rot_wheel), axis=1)
    # frame = rotate_image(frame, angle_v[i])
    cv.imshow("frame", frame)
    cv.imshow("wheel", rot_wheel)
    i += 1

    key = cv.waitKey()

    if key == 113:
        cv.destroyAllWindows()
        break


print(i)
cv.destroyAllWindows()

NameError: name 'vid' is not defined

In [22]:
class FrameWithAngle(object):
    def __init__(self, frame, angle):
        self.frame = frame
        self.angle = angle


# Pre process data

base_dir = "/media/nevin/Trash Games1/Downloads/comma2k19/extracted/Chunk_1"

dirs = os.listdir(base_dir)

for _dir in dirs:
    project_dir = os.path.join(base_dir, _dir)
    segments = os.listdir(project_dir)

    for segment in segments:
        seg_dir = os.path.join(project_dir, segment)
        video_path = os.path.join(seg_dir, "video.hevc")

        frame_times = np.load(os.path.join(seg_dir, "global_pose/", "frame_times"))
        angle_t = np.load(os.path.join(seg_dir, "processed_log/", "CAN/", "steering_angle/", "t"))
        angle_v = np.load(os.path.join(seg_dir, "processed_log/", "CAN/", "steering_angle/", "value"))

        video = cv.VideoCapture(video_path)

        frames = []

        print(f"reading {video_path}")

        if video.isOpened() == False:
            print(f"Failed to open video {video_path}")
            continue

        i = 0

        while video.isOpened():
            status, frame = video.read()

            if status == False:
                break

            # find the nearest frame which matches the angle input
            rot_idx = find_nearest(angle_t, frame_times[i])
            angle = angle_v[rot_idx]

            frames.append(FrameWithAngle(frame, angle))
            i += 1
        

        print(frames)
        print(sys.getsizeof(frames))
        break

    # print(files)

# print(dirs)

reading /media/nevin/Trash Games1/Downloads/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b_2018-07-27--06-03-57/10/video.hevc
[<__main__.FrameWithAngle object at 0x7f3956048f10>, <__main__.FrameWithAngle object at 0x7f395603d400>, <__main__.FrameWithAngle object at 0x7f38fc6fe9d0>, <__main__.FrameWithAngle object at 0x7f38fc6feb50>, <__main__.FrameWithAngle object at 0x7f38fc6fe700>, <__main__.FrameWithAngle object at 0x7f38fc6fe9a0>, <__main__.FrameWithAngle object at 0x7f38fc6febe0>, <__main__.FrameWithAngle object at 0x7f38fc6febb0>, <__main__.FrameWithAngle object at 0x7f38fc6fea00>, <__main__.FrameWithAngle object at 0x7f38fc6feb20>, <__main__.FrameWithAngle object at 0x7f38fc6feca0>, <__main__.FrameWithAngle object at 0x7f38fc6fecd0>, <__main__.FrameWithAngle object at 0x7f38fc6fef10>, <__main__.FrameWithAngle object at 0x7f38fc6fefd0>, <__main__.FrameWithAngle object at 0x7f38fc6fee80>, <__main__.FrameWithAngle object at 0x7f38fc6fefa0>, <__main__.FrameWithAngle object at 0x7f38fc

KeyboardInterrupt: 

In [ ]:
class DriveDataset(Dataset):
    def __init__(self):
        pass